In [ ]:
import scrapy
from scrapy import Spider
from scrapy.exporters import CsvItemExporter
import csv

from .. import items


class MusinsaSpider(Spider):
	# 스파이더 이름(실행)
    name = "test"

    def start_requests(self):
        url = "https://www.musinsa.com/categories/item/001004"
        # 상의 - 후드티
        yield scrapy.Request(url, self.parse_start)

    # 상품 상세 페이지 url로 request
    def parse_start(self, response):
        product_url = "https://www.musinsa.com/app/goods/"
        data_no = response.xpath('//li[@class="li_box"]/@data-no').getall()
        for d in data_no:
            url = product_url + d
            yield scrapy.Request(url, self.parse_items)

    # 상품 상세 페이지에 있는 정보 크롤링
    def parse_items(self, response):
        item = items.MusinsaItem()
        item['product_name'] = response.xpath('//span[@class="product_title"]/em/text()').get().strip()
        item['brand'] = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[1]/p[2]/strong/a/text()').get()
        item['product_id'] = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[1]/p[2]/strong/text()')[
            1].get().strip()
        item['season'] = response.xpath(
            '//*[@id="product_order_info"]/div[1]/ul/li[2]/p[2]/strong/text()').get().strip().replace(" ", "")
        item['sex'] = response.xpath('//*[@id="product_order_info"]/div[1]/ul/li[2]/p[2]/span[2]/span/text()').get()
        item['like'] = response.xpath('//*[@id="product-top-like"]/p[2]/span/text()').get()
        item['price'] = response.xpath('//*[@id="goods_price"]/del/text()').get()
        item['product_img'] = response.xpath('//*[@id="bigimg"]/@src').get()
        item['product_info'] = response.xpath('//*[@id="detail_view"]/div[1]/div/img/@src').getall()

        yield item

    def close(self, reason):
        # This method is called when the spider is closed
        csv_file = open('musinsa_data_hoodie.csv', 'wb')
        csv_exporter = CsvItemExporter(csv_file)
        csv_exporter.fields_to_export = ['product_name', 'brand', 'product_id', 'season', 'sex', 'like', 'price', 'product_img', 'product_info']
        csv_exporter.start_exporting()

        for item in self.parse_items(response=None):
            csv_exporter.export_item(item)

        csv_exporter.finish_exporting()
        csv_file.close()

In [ ]:
import scrapy
from scrapy import Spider
from scrapy.exporters import CsvItemExporter
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

from items import MusinsaItem


class MusinsaSpider(Spider):
    name = "test"

    def start_requests(self):
        url = "https://www.musinsa.com/categories/item/001004"
        # 상의 - 후드티
        yield scrapy.Request(url, self.parse_start)

    def parse_start(self, response):
        product_url = "https://www.musinsa.com/app/goods/"
        data_no = response.xpath('//li[@class="li_box"]/@data-no').getall()
        
        chrome_options = Options()
        chrome_options.add_argument("--headless")  # Run Chrome in headless mode (without UI)
        driver = webdriver.Chrome(options=chrome_options)

        for d in data_no:
            url = product_url + d
            driver.get(url)
            yield scrapy.Request(url, self.parse_items, meta={'driver': driver})

    def parse_items(self, response):
        driver = response.meta['driver']

        item = MusinsaItem()
        item['product_name'] = driver.find_element_by_xpath('//span[@class="product_title"]/em').text.strip()
        item['brand'] = driver.find_element_by_xpath('//*[@id="product_order_info"]/div[1]/ul/li[1]/p[2]/strong/a').text
        item['product_id'] = driver.find_element_by_xpath('//*[@id="product_order_info"]/div[1]/ul/li[1]/p[2]/strong[2]').text.strip()
        item['season'] = driver.find_element_by_xpath('//*[@id="product_order_info"]/div[1]/ul/li[2]/p[2]/strong').text.strip().replace(" ", "")
        item['sex'] = driver.find_element_by_xpath('//*[@id="product_order_info"]/div[1]/ul/li[2]/p[2]/span[2]/span').text
        item['like'] = driver.find_element_by_xpath('//*[@id="product-top-like"]/p[2]/span').text
        item['price'] = driver.find_element_by_xpath('//*[@id="goods_price"]/del').text
        item['product_img'] = driver.find_element_by_xpath('//*[@id="bigimg"]').get_attribute('src')
        item['product_info'] = [img.get_attribute('src') for img in driver.find_elements_by_xpath('//*[@id="detail_view"]/div[1]/div/img')]

        yield item